In [ ]:
!pip install yfinance
!pip install --upgrade pandas
!pip install --upgrade pandas-datareader

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
import yfinance as yf
from scipy.stats import norm
from datetime import datetime

In [2]:
start = datetime(2017,1,1)
end = datetime(2029,1,1)
close = yf.download('XLK',start,end)['Close']

log_returns = pd.Series(np.log(close.drop(close.index[-1]).values/close.drop(close.index[0]).values),
          index=close.index[1:],
          name='Logarithmic Returns')
#252 anual
std_mensual = np.sqrt(21)*log_returns.std()
var_mensual = std_mensual**2


[*********************100%%**********************]  1 of 1 completed


In [16]:
var_mensual

0.005390794284686305

In [34]:
r = 0.06
R = np.exp(r/12)

#X = R+np.sqrt(-R**2 + R + var_mensual)
#Y = R-np.sqrt(-R**2 + R + var_mensual)

#X = R+np.sqrt(var_mensual)
#Y = R-np.sqrt(var_mensual)

#aqui de hecho X y Y no son los del documento sino que son los factores como tal;
#es decir X = exp(X*), Y = exp(Y*) donde X*,Y* son los del documento
X = np.exp(r/12 + np.sqrt(var_mensual))
Y = np.exp(r/12 - np.sqrt(var_mensual))

In [22]:
Y

0.9338662721194856

In [23]:
X

1.0815790196510437

In [24]:
#up queda como izquierda, down a la derecha
def arbolBinomial(down,up,T=12,S0=60):
    dominio = np.arange(0,T+1,1)

    x = dominio
    y = [[S0]]
    for i in range(1,len(x)):
      past = y[-1]
      present = []
      for value in past:
        #produce new values
        present.append(value*down)
      present.append(past[-1]*up)
      y.append(present)
    return pd.DataFrame(y)
s=arbolBinomial(X,Y)

In [32]:
#arbol del subyacente
s

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,60.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,64.894741,56.031976,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,70.188791,60.603010,52.326373,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,75.914723,65.546944,56.595107,48.865835,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,82.107772,70.894200,61.212080,52.852262,45.634155,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,88.806044,76.677679,66.205702,57.163897,49.356945,42.616198,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,96.050753,82.932969,71.606698,61.827272,53.383436,46.092786,39.797830,NaN,NaN,NaN,NaN,NaN,NaN
7,103.886480,89.698559,77.448302,66.871080,57.738404,49.852990,43.044498,37.165851,NaN,NaN,NaN,NaN,NaN
8,112.361437,97.016080,83.766459,72.326357,62.448646,53.919948,46.556026,40.197805,34.707935,NaN,NaN,NaN,NaN
9,121.527773,104.930556,90.600045,78.226671,67.543146,58.318685,50.354021,43.477102,37.539374,32.412570,NaN,NaN,NaN


In [33]:
def put(S,k):
  return max(k-S,0)

def call(S,k):
  return max(S-k,0)

def callDig(S,k):
  if S>=k:
    ret = 1
  else:
    ret = 0
  return ret

def putDig(S,k):
  return max(k-S,0)/(k-S)

def chooser(s1, s2):
  return max(s1,s2)

def calcularAnterior(V1,V2,p,r):
  V = ((p)*V1+(1-p)*V2)/(r)
  #V = (V1+V2)/2
  return V

def backPropagationEurop(v,p,r):
  v = v.copy(deep=True)
  for k in range(1,v.shape[0]):
      #i itera desde v.shape[0]-2 hasta 0 osea penultimo periodo hasta primer periodo
      i = v.shape[0] - k - 1
      for j in range(0,i+1+v.shape[1]-v.shape[0]):
        #j itera desde 0 hasta la fila actual MAS la diferencia entre columnas y filas
        #la diferencia entre columnas y filas es la informacion de hasta donde hacia atras en el tiempo se esta realizando el modelo binomial
        #es decir si S.shape[0]=6 y S.shape[1] = 12 es decir que esto de alguna forma vino de un arbol binomial de 12 periodos pero solo se esta viendo desde el periodo 12 hasta el 6
        #creo que esta forma de escribir el algorimto solo admite slices conexos (y probablemnete solo del final hacia atras aunque no estoy seguro) del arbol binomial original.
        v.iloc[i,j] = calcularAnterior(v.iloc[i+1,j],v.iloc[i+1,j+1],p,r)

  return v

def backPropagationAmer(v,p,r,strike,funcionPago):
  v = v.copy(deep=True)
  for k in range(1,v.shape[0]):
    #i itera desde v.shape[0]-2 hasta 0 osea penultimo periodo hasta primer periodo
    i = v.shape[0] - k - 1
    for j in range(0,i+1+v.shape[1]-v.shape[0]):
      if funcionPago==call and funcionPago(v.iloc[i,j],strike) > calcularAnterior(v.iloc[i+1,j],v.iloc[i+1,j+1],p,r):
        #si esto imprime es porque la valoracion del anterior dados los dos siguientes quedo mal, puede ser uso incorrecto de tasa de r
        print((i,j),'ejerc-valor:',100*(funcionPago(v.iloc[i,j],strike)-calcularAnterior(v.iloc[i+1,j],v.iloc[i+1,j+1],p,r))/funcionPago(v.iloc[i,j],strike))
      v.iloc[i,j] = max(funcionPago(v.iloc[i,j],strike), calcularAnterior(v.iloc[i+1,j],v.iloc[i+1,j+1],p,r))

  return v

def valoracionPago_1Sub1Str(s,strike,funcionPago,earlyExpiry=None):
  v = s.copy(deep=True)
  if earlyExpiry==None:
    earlyExpiry = v.shape[0]-1
  for j in range(v.shape[1]):
    v.iloc[earlyExpiry,j] = funcionPago(v.iloc[v.shape[0]-1,j],strike)

  return v


def valoracionPago_2Sub(v,s1,s2,funcionPago):
  v = v.copy(deep=True)
  for j in range(v.shape[1]):
    v.iloc[v.shape[0]-1,j] = funcionPago(s1.iloc[s1.shape[0]-1,j],s2.iloc[s2.shape[0]-1,j])

  return v


def valoracionOpcion(S,p,r,strike,funcionPago=call,tipo='europea'):
  v = S.copy(deep=True)
  #primero se valora el pago final sin fecha anticipada de expiracion
  v = valoracionPago_1Sub1Str(v,strike,funcionPago)
  #se valora el tiempo hacia atras
  if tipo=='europea':
    v = backPropagationEurop(v,p,r)
  elif tipo=='americana':
    v = backPropagationAmer(v,p,r,strike,funcionPago)
  return v



In [27]:
def valoracionChooser(S,chooserExpiry,p,R,strikeOpc,tipoOpc='europea'):

  c = S.copy(deep=True).iloc[:chooserExpiry+1,:]

  callOp = valoracionOpcion(s,p,R,strikeOpc,call,tipoOpc).iloc[:chooserExpiry+1,:]
  putOp = valoracionOpcion(s,p,R,strikeOpc,put,tipoOpc).iloc[:chooserExpiry+1,:]

  c = valoracionPago_2Sub(c,callOp,putOp,chooser)

  c = backPropagationEurop(c,0.5,R)


  return c


In [28]:
#punto 2a
valoracionOpcion(s,0.5,R,60,put,'americana')


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,4.258395,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.562406,5.997074,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.362128,3.788372,8.265898,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.603687,2.134225,5.480496,11.134165,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.200066,1.013360,3.276487,7.739448,14.365845,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.037650,0.364488,1.672391,4.913429,10.643055,17.383802,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0.000000,0.075677,0.656953,2.704595,7.171520,13.907214,20.202170,NaN,NaN,NaN,NaN,NaN,NaN
7,0.000000,0.000000,0.152112,1.168380,4.267925,10.147010,16.955502,22.834149,NaN,NaN,NaN,NaN,NaN
8,0.000000,0.000000,0.000000,0.305748,2.042724,6.535911,13.443974,19.802195,25.292065,NaN,NaN,NaN,NaN
9,0.000000,0.000000,0.000000,0.000000,0.614562,3.491366,9.645979,16.522898,22.460626,27.587430,NaN,NaN,NaN


In [35]:
#punto 2b
valoracionOpcion(s,0.5,R,70,callDig,'europea')

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.364658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.473243,0.259729,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.592661,0.358569,0.163493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.713264,0.477999,0.242734,0.085890,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.821925,0.611753,0.349037,0.138864,0.033778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.905255,0.746835,0.482803,0.218770,0.060350,0.007544,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0.955282,0.864303,0.636855,0.333591,0.106142,0.015163,0.0,NaN,NaN,NaN,NaN,NaN,NaN
7,0.975310,0.944831,0.792439,0.487655,0.182871,0.030478,0.0,0.0,NaN,NaN,NaN,NaN,NaN
8,0.980199,0.980199,0.918936,0.673887,0.306312,0.061262,0.0,0.0,0.0,NaN,NaN,NaN,NaN
9,0.985112,0.985112,0.985112,0.861973,0.492556,0.123139,0.0,0.0,0.0,0.0,NaN,NaN,NaN


In [30]:
#punto 2c
valoracionChooser(s,6,0.5,R,65,'europea')

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,10.509639,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11.148644,9.975993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13.222690,9.186363,10.865632,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,16.880211,9.697728,8.767092,13.073100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,21.945770,11.983877,7.508798,10.113277,16.163981,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,27.875118,16.236429,7.851464,7.241408,13.086533,19.403473,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,34.555933,21.473752,11.161877,4.619762,9.935649,16.368609,22.632858,NaN,NaN,NaN,NaN,NaN,NaN
